In [1]:
# Connecting to clickhouse on remote server
import clickhouse_connect
clickhouse_client = clickhouse_connect.get_client(host='localhost')
clickhouse_client.command('SET max_threads = 8;')

In [2]:
import heapq
def moving_average(arr, window_size):
    weights = np.ones(window_size) / window_size
    return np.convolve(arr, weights, mode='valid')

def get_throughput(series):
    W = 10
    arr = series.to_numpy()
    mv_avg = moving_average(arr, W)
    largest_numbers = heapq.nlargest(W, mv_avg)
    return sum(largest_numbers) / W

In [3]:
test_ids = [
    '31f73544dae74716ba3af75330d02301',
    'b2a8430073e847cc86b19f940bd96f30',
    'de390cac8de345d997e8376a7e36eb5f',
    '71cbc372fc174c1ea07389b2d30bd4a7'
]

In [14]:
import os
import glob
wgn_files = glob.glob('dataset/df_wgn_*.parquet')
print(wgn_files)

['dataset/df_wgn_2_de390cac8de345d997e8376a7e36eb5f_20_-12.2.parquet', 'dataset/df_wgn_3_71cbc372fc174c1ea07389b2d30bd4a7_15_-7.6.parquet', 'dataset/df_wgn_2_de390cac8de345d997e8376a7e36eb5f_3_-22.0.parquet', 'dataset/df_wgn_2_de390cac8de345d997e8376a7e36eb5f_9_-16.0.parquet', 'dataset/df_wgn_3_71cbc372fc174c1ea07389b2d30bd4a7_10_-8.6.parquet', 'dataset/df_wgn_3_71cbc372fc174c1ea07389b2d30bd4a7_12_-8.2.parquet', 'dataset/df_wgn_2_de390cac8de345d997e8376a7e36eb5f_1_-24.0.parquet', 'dataset/df_wgn_2_de390cac8de345d997e8376a7e36eb5f_13_-13.6.parquet', 'dataset/df_wgn_2_de390cac8de345d997e8376a7e36eb5f_21_-12.0.parquet', 'dataset/df_wgn_1_b2a8430073e847cc86b19f940bd96f30_0_-32.0.parquet', 'dataset/df_wgn_0_31f73544dae74716ba3af75330d02301_6_-40.0.parquet', 'dataset/df_wgn_0_31f73544dae74716ba3af75330d02301_0_-100.0.parquet', 'dataset/df_wgn_3_71cbc372fc174c1ea07389b2d30bd4a7_7_-9.2.parquet', 'dataset/df_wgn_0_31f73544dae74716ba3af75330d02301_4_-60.0.parquet', 'dataset/df_wgn_0_31f73544dae7

In [17]:
import numpy as np
import pandas as pd
import pickle
import datetime

dataset = pd.DataFrame()
# ignores = [-90, -70, -50, -39, -37, -35, -33, -31, -29, -27, -25, -23, -21, -19, -17, -15, -13.8, -13.4, -12.8, -12.4, -11.8, -11.4, -10.8, -10.4, -9.8, -9.4, -8.8, -8.4, -7.8, -7.4]
ignores = []

for test_id_idx, test_id in enumerate(test_ids):    
    print(f'------ Test ID {test_id}')
    query = f"""
        SELECT *
        FROM wgn
        WHERE test_id = '{test_id}' AND direction = 0
        ORDER BY timestamp
    """
    df_wgn =  clickhouse_client.query_df(query)
    # print(df_wgn)
    # noise = df_wgn['noise_level'].to_numpy()
    # print(noise)

    # Group by noise_level and get the start (min) and stop (max) timestamps
    df_noise_time_ranges = df_wgn.groupby('noise_level').agg(
        begin_time=('timestamp', 'min'),
        end_time=('timestamp', 'max'),
        max_throughput=('throughput_mbps', get_throughput),
    ).reset_index()
    df_noise_time_ranges = df_noise_time_ranges.sort_values('noise_level')
    # print(df_noise_time_ranges)

    for noise_id_idx, noise_record in df_noise_time_ranges.iterrows():
        if f"dataset/df_wgn_{test_id_idx}_{test_id}_{noise_id_idx}_{noise_record.noise_level}.parquet" in wgn_files:
            continue
        if noise_record.noise_level in ignores:
            continue

        print(f" - Noise level {noise_record.noise_level} between {noise_record.begin_time} {noise_record.end_time}")

        while True:
            try:
                print("wgn")
                query = f"""
                    SELECT *
                    FROM wgn
                    WHERE timestamp BETWEEN '{noise_record.begin_time}' AND '{noise_record.end_time}'
                    ORDER BY timestamp DESC
                """
                df_wgn =  clickhouse_client.query_df(query)

                print("mac")
                query = f"""
                SELECT * FROM MAC_KPIs_2 
                WHERE TsTaiNs BETWEEN '{noise_record.begin_time}' AND '{noise_record.end_time}'
                ORDER BY TsTaiNs DESC
                """
                # print(query)
                df_mac = clickhouse_client.query_df(query)
                # print(df_mac)
        
                print("fapi")
                query = f"""
                SELECT * FROM fapi 
                WHERE TsTaiNs BETWEEN '{noise_record.begin_time}' AND '{noise_record.end_time}'
                ORDER BY TsTaiNs DESC
                """
                # print(query)
                df_fapi = clickhouse_client.query_df(query)
                # print(df_fapi)
                # After query
                df_fapi['index_TsTaiNs'] = df_fapi['TsTaiNs']  # Preserve original timestamps
                df_fapi.set_index('index_TsTaiNs', inplace=True)
                df_fapi = df_fapi.resample('1s').first()
                # print(df_fapi)
        
                print("fh")
                # Extract TsTaiNs values from df_fapi
                ts_tai_ns_values = df_fapi['TsTaiNs'].tolist()
                ts_tai_ns_values_str = ','.join(f"'{ts}'" for ts in ts_tai_ns_values)  # Format as SQL-compatible string
                # Build the query
                query = f"""
                SELECT TsTaiNs,fhData
                FROM fh
                WHERE TsTaiNs BETWEEN '{noise_record.begin_time}' AND '{noise_record.end_time}'
                  AND TsTaiNs IN ({ts_tai_ns_values_str})
                ORDER BY TsTaiNs DESC
                """
                # print(query)
                df_fh = clickhouse_client.query_df(query)
                # print(df_fh)

                df_wgn.to_parquet(f"dataset/df_wgn_{test_id_idx}_{test_id}_{noise_id_idx}_{noise_record.noise_level}.parquet", index=False)
                df_mac.to_parquet(f"dataset/df_mac_{test_id_idx}_{test_id}_{noise_id_idx}_{noise_record.noise_level}.parquet", index=False)
                df_fapi.to_parquet(f"dataset/df_fapi_{test_id_idx}_{test_id}_{noise_id_idx}_{noise_record.noise_level}.parquet", index=False)
                df_fh.to_parquet(f"dataset/df_fh_{test_id_idx}_{test_id}_{noise_id_idx}_{noise_record.noise_level}.parquet", index=False)
                break
            except Exception as e:
                print(e)
    #     break
    # break

------ Test ID 31f73544dae74716ba3af75330d02301
 - Noise level -90.0 between 2025-04-14 06:35:45 2025-04-14 06:43:00
wgn
mac
fapi
fh
 - Noise level -70.0 between 2025-04-14 06:50:47 2025-04-14 06:58:02
wgn
mac
fapi
fh
 - Noise level -50.0 between 2025-04-14 07:05:50 2025-04-14 07:13:04
wgn
mac
fapi
fh
 - Noise level -39.0 between 2025-04-14 07:20:52 2025-04-14 07:28:07
wgn
mac
fapi
fh
 - Noise level -37.0 between 2025-04-14 07:35:55 2025-04-14 07:43:10
wgn
mac
fapi
fh
 - Noise level -35.0 between 2025-04-14 07:50:59 2025-04-14 07:58:14
wgn
mac
fapi
fh
 - Noise level -33.0 between 2025-04-14 08:06:02 2025-04-14 08:13:16
wgn
mac
fapi
fh
------ Test ID b2a8430073e847cc86b19f940bd96f30
 - Noise level -31.0 between 2025-04-14 08:33:01 2025-04-14 08:40:16
wgn
mac
fapi
fh
 - Noise level -29.0 between 2025-04-14 08:48:04 2025-04-14 08:55:19
wgn
mac
fapi
fh
 - Noise level -27.0 between 2025-04-14 09:03:07 2025-04-14 09:10:21
wgn
mac
fapi
fh
 - Noise level -25.0 between 2025-04-14 09:18:09 2025-